In [20]:
import requests
import pandas as pd

import re

from bs4 import BeautifulSoup

In [3]:
# site data
URL = "https://kreyol.com/dictionary/"
pages = ["Aa.html", "Bb.html", "Cc.html", "Dd.html", "Ee.html", 
        "Ff.html", "Gg.html", "Hh.html", "Ii.html", "Jj.html", 
        "Kk.html","Ll.html", "Mm.html", "Nn.html", "Oo.html",
        "Pp.html", "Qq.html", "Rr.html", "Ss.html", "Tt.html",
        "Uu.html", "Vv.html", "Ww.html", "Xx.html", "Yy.html",
        "Zz.html"]

In [27]:
x = URL + pages[0]
print(x)
page = requests.get(URL + "Aa.html")
# print(page.text)

soup = BeautifulSoup(page.content, "html.parser")

y = soup.find_all("p")
# print(y)
words = []
count = 0
for element in y:
    # print(type(element))
    # print(type(element.text))
    if "/" in element.text and "href" not in element.text:
#         print(count, "-", element)
        words.append(element.text)
    count += 1
    
print(len(words))
# print(words)



https://kreyol.com/dictionary/Aa.html
3


In [55]:
# a = words[0].split("\n")
# print(a[0] == "")
# print(a)
test = words[0]
found = re.search("\([^\(\)]*\n[^\(]*\)",test)

while found:
    start, end = found.span()
    print(start, ", ", end)
    print(test[start:end])
    ran = test[start:end]
    ind = ran.index("\n")
    print(ind)
    temp = list(test)
    print("character at start + ind:", temp[start + ind], "neat!")
    temp[start + ind] = ""
#     print(temp[end-2])
    test = "".join(temp)
    found = re.search("\([^\(\)]*\n[^\(]\)",test)
        
a = test.split("\n")
        
wordlist_test = pd.DataFrame(columns=["word", "creole_word", "creole name"])
flip = False

for entry in a:
    if entry != "":  # check if the itme isn't an empty string
        if not flip and "Kreyol / English" in entry: # after this page on the site the side the english is on and the side the creole is on flips
            flip = True
            continue
        if "/" in entry:
            text = ["", "", "hatian"]
            if not flip:
                temp = entry.split("/")
#                 print(temp)
                text[0] = temp[0].strip()
                text[1] = temp[1].strip()
            else:
                temp = entry.split("/")
#                 print(temp)
                text[0] = temp[1].strip()
                text[1] = temp[0].strip()

            wordlist_test.loc[len(wordlist_test.index)] = text

377 ,  423
(loathe, to regard with extreme 
  repugnance)
32
character at start + ind: 
 neat!


In [56]:
wordlist_test

,word,creole_word,creole name
0,"a, an",youn,hatian
1,aback,pa sipriz (taken by surprise),hatian
2,abandon,"abandone, kite",hatian
3,abase,"abese (to lower in rank-office-prestige), ~dim...",hatian
4,abash,demoralize (to destroy the self-confidence of),hatian
...,...,...,...
738,Haiti,Ayiti (Peyi Dayiti),hatian
739,Souteast Asia,Azi di Disès,hatian
740,East Asia,Azi de Lès,hatian
741,haven,azil,hatian


In [57]:
wordlist_test.loc[9, "creole_word"]

'deteste (loathe, to regard with extreme   repugnance), rayi (hate)'

In [88]:
wordlist = pd.DataFrame(columns=["word", "creole_word", "creole name"])
prev = False # keep track of what the second cell was
for site_page in pages: # loop through all the alphabet pages 
    x = URL + site_page
    print(x)
    
    page = requests.get(x)
    

    soup = BeautifulSoup(page.content, "html.parser") # pull the data from the site
    y = soup.find_all("p")

    words = []
    count = 0
    for element in y: # go through all the elements soup got and only save those elements that have a slash - the reason that all definitions have slashes in them 
        if "/" in element.text and "href" not in element.text:
            words.append(element.text)
        count += 1


    flip = False
    words_split = []
#     print(words[0])
    for a in words:
        # regular expressions
        test = a
        found = re.search("\([^\(\)]*\n[^\(]*\)",test)

        while found:
            start, end = found.span()
            print(start, ", ", end)
            print(test[start:end])
            ran = test[start:end]
            ind = ran.index("\n")
            print(ind)
            temp = list(test)
            print("character at start + ind:", temp[start + ind], "neat!")
            temp[start + ind] = ""
        #     print(temp[end-2])
            test = "".join(temp)
            found = re.search("\([^\(\)]*\n[^\(]\)",test)
        
        z = test.split("\n") # split the huge string grabbed by the remaining newline characters
        words_split.extend(z)

    for entry in z: # loop through every entry that was found
        if entry != "":  # check if the itme isn't an empty string
            if not flip and ("Kreyol / English" in entry or "kreyol - english" in entry): # after this page on the site the side the english is on and the side the creole is on flips
                flip = True
                continue
            if "/" in entry and "Kreyol pronunciation" != entry:
                text = ["", "", "hatian"]
                if not flip:
                    temp = entry.split("/")

                    text[0] = temp[0].strip() # english
                    
                    # in case there is a comma in the parentesis in the creole words
                    test = temp[1]
                    found = re.search("\([^\(\)]*,[^\(]*\)",test)
                    
                    if found:
                        while found:
                            start, end = found.span()
#                             print(start, ", ", end)
#                             print(test[start:end])
                            ind = test[start:end].index(",")
#                             print(ind)
                            temp = list(test)
                            print("character at start + ind:", temp[start + ind], "neat!")
                            temp[start + ind] = "%"
                        #     print(temp[end-2])
                            test = "".join(temp)
                            found = re.search("\([^\(\)]*,[^\(]*\)",test)
                    
                        creole_words = test.split(",")
                        for ii in range(len(creole_words)):
                            if "%" in creole_words[ii]:
                                creole_words[ii] = creole_words[ii].replace("%", ",")
                                
                        text[1] = creole_words
                    else:  # if there aren't extra commas in the creole langauge half, then just append it to the list                    
                        text[1] = temp[1].strip().split(",") # Creole
                                       
                else:
                    temp = entry.split("/")
                    text[0] = temp[1].strip() # English
                    
                    
                    # in case there is a comma in the parentesis in the creole words
                    test = temp[0]
                    found = re.search("\([^\(\)]*,[^\(]*\)",test)
                    
                    if found:
                        while found:
                            start, end = found.span()
#                             print(start, ", ", end)
#                             print(test[start:end])
                            ind = test[start:end].index(",")
#                             print(ind)
                            temp = list(test)
                            print("character at start + ind:", temp[start + ind], "neat!")
                            temp[start + ind] = "%"
                        #     print(temp[end-2])
                            test = "".join(temp)
                            found = re.search("\([^\(\)]*,[^\(]*\)",test)
                    
                        creole_words = test.split(",")
                        for ii in range(len(creole_words)):
                            if "%" in creole_words[ii]:
                                creole_words[ii] = creole_words[ii].replace("%", ",")
                                
                        text[1] = creole_words
                    else:  # if there aren't extra commas in the creole langauge half, then just append it to the list                    
                        text[1] = temp[0].strip().split(",") # Creole
                    
                # get rid of the parts of speech
                if "(v)" in text[0]:
                    text[0] = text[0].replace("(v)", "")
                    text[0] = text[0].strip()

                if "(n)" in text[0]:
                    text[0] = text[0].replace("(n)", "")
                    text[0] = text[0].strip()

                if "(adj)" in text[0]:
                    text[0] = text[0].replace("(adj)", "")

                prev = flip
                
                wordlist.loc[len(wordlist.index)] = text
            elif "/" not in entry and "Kreyol pronunciation" != entry:
                wordlist.iloc[len(wordlist.index)-1, 1].append(entry)


https://kreyol.com/dictionary/Aa.html
377 ,  423
(loathe, to regard with extreme 
  repugnance)
32
character at start + ind: 
 neat!
372 ,  418
(loathe, to regard with extreme 
  repugnance)
32
character at start + ind: 
 neat!
369 ,  415
(loathe, to regard with extreme 
  repugnance)
32
character at start + ind: 
 neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , neat!
character at start + ind: , nea

In [89]:
# wordlist

In [90]:
# wordlist.head(10)

In [91]:
wordlist.loc[9, "creole_word"]

[' deteste (loathe, to regard with extreme   repugnance)', ' rayi (hate)']

In [92]:
wordlist.to_csv("hatian_creole_dictionary_v3.csv")